# Выбор локации для скважины

**Описание проекта.**

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.


Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

**Цель исследования** - Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

**Описание данных.**
 - `id` — уникальный идентификатор скважины
 - `f0`,`f1`,`f2` — три признака точек (неважно, что они означают, но сами признаки значимы)
 - `product ` — объём запасов в скважине (тыс. баррелей)

## Загрузка и подготовка данных

In [93]:
import pandas as pd
import numpy as np
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1500)
warnings.simplefilter("ignore")

In [94]:
try:
  df0 = pd.read_csv('/DS/Yandex_Practikum/Projects/datasets/geo_data_0.csv')
except:
  df0 = pd.read_csv('/datasets/geo_data_0.csv')
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [95]:
try:
  df1 = pd.read_csv('/DS/Yandex_Practikum/Projects/datasets/geo_data_1.csv')
except:
  df1 = pd.read_csv('/datasets/geo_data_1.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [96]:
try:
  df2 = pd.read_csv('/DS/Yandex_Practikum/Projects/datasets/geo_data_2.csv')
except:
  df2 = pd.read_csv('/datasets/geo_data_2.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [97]:
df0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [98]:
features0 = df0.drop(['product','id'],axis = 1)
target0 = df0['product'] 

In [99]:
features1 = df1.drop(['product','id'],axis = 1)
target1 = df1['product'] 

In [100]:
features2 = df2.drop(['product','id'],axis = 1)
target2 = df2['product'] 

In [101]:
state = np.random.RandomState(12345)

In [102]:
def split(features,target):
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=state)
    return features_train, features_valid, target_train, target_valid

In [103]:
scaler = StandardScaler()
def numeric(features_train,features_valid):
    scaler.fit(features_train)

    features_train_numeric = scaler.transform(features_train)
    features_valid_numeric = scaler.transform(features_valid)
    
    return features_train_numeric, features_valid_numeric

In [104]:
features_train0, features_valid0, target_train0, target_valid0 = split(features0,target0)
features_train1, features_valid1, target_train1, target_valid1 = split(features1,target1)
features_train2, features_valid2, target_train2, target_valid2 = split(features2,target2)

In [105]:
features_train_numeric0, features_valid_numeric0 = numeric(features_train0,features_valid0)
features_train_numeric1, features_valid_numeric1 = numeric(features_train1,features_valid1)
features_train_numeric2, features_valid_numeric2 = numeric(features_train2,features_valid2)

## Обучение и проверка модели

In [106]:
model = LinearRegression()
def model_linear_regression(features_train,target_train,features_valid,target_valid,features):
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    result = mean_squared_error(target_valid,predicted_valid)
    rmse = result ** 0.5
    predicted = model.predict(features)
    return predicted_valid, rmse, predicted

In [107]:
predicted_valid0, rmse0, predicted_values0 = model_linear_regression (features_train0, target_train0, features_valid0, target_valid0, features0)
predicted_valid1, rmse1, predicted_values1 = model_linear_regression (features_train1, target_train1, features_valid1, target_valid1, features1)
predicted_valid2, rmse2, predicted_values2 = model_linear_regression (features_train2, target_train2, features_valid2, target_valid2, features2)

In [108]:
print('Среднее значение предсказанного сырья 0 региона:',predicted_valid0.mean(),'   rmse:',rmse0)
print('Среднее значение предсказанного сырья 1 региона:',predicted_valid1.mean(),'   rmse:',rmse1)
print('Среднее значение предсказанного сырья 2 региона:',predicted_valid2.mean(),'     rmse:',rmse2)

Среднее значение предсказанного сырья 0 региона: 92.59256778438035    rmse: 37.5794217150813
Среднее значение предсказанного сырья 1 региона: 68.76995145799754    rmse: 0.8897367737680649
Среднее значение предсказанного сырья 2 региона: 95.087528122523      rmse: 39.958042459521614


In [109]:
predicted_valid0, rmse0, predicted_values0 = model_linear_regression (features_train_numeric0, target_train0, features_valid_numeric0, target_valid0, features0)
predicted_valid1, rmse1, predicted_values1 = model_linear_regression (features_train_numeric1, target_train1, features_valid_numeric1, target_valid1, features1)
predicted_valid2, rmse2, predicted_values2 = model_linear_regression (features_train_numeric2, target_train2, features_valid_numeric2, target_valid2, features2)

In [110]:
print('Среднее значение предсказанного сырья 0 региона:',predicted_valid0.mean(),'   rmse:',rmse0)
print('Среднее значение предсказанного сырья 1 региона:',predicted_valid1.mean(),'   rmse:',rmse1)
print('Среднее значение предсказанного сырья 2 региона:',predicted_valid2.mean(),'     rmse:',rmse2)

Среднее значение предсказанного сырья 0 региона: 92.59256778438035    rmse: 37.5794217150813
Среднее значение предсказанного сырья 1 региона: 68.76995145799754    rmse: 0.889736773768065
Среднее значение предсказанного сырья 2 региона: 95.087528122523      rmse: 39.958042459521614


Из представленных результатов видно, что предсказанные результаты для первого и третьего региона с большой ошибкой не зависимо от использованяи стандартизации признаков.

In [111]:
def df_predicted(data,predicted_values):
    df_predicted = data.copy()
    df_predicted['product_predicted'] = predicted_values
    return df_predicted

In [112]:
df0_predicted = df_predicted(df0,predicted_values0)
df1_predicted = df_predicted(df1,predicted_values1)
df2_predicted = df_predicted(df2,predicted_values2)

## Подготовка к расчёту прибыли

In [113]:
NUMBER_OF_BEST_POINTS = 200
PRICE_PER_PRODUCT = 450 * 10 ** 3
BUDGET = 10 ** 10

In [114]:
PRODUCT_SUM = BUDGET / PRICE_PER_PRODUCT
PRODUCT_SUM

22222.222222222223

Безубыточное количество продукта на разработку каждого региона равно 22222.222222222223 (тыс. баррелей). Т.е. в 200 лучших точек для разработки должно быть в сумме не менее 22222.222222222223 продукта

In [115]:
PRODUCT_PER_POINT = PRODUCT_SUM / NUMBER_OF_BEST_POINTS
PRODUCT_PER_POINT

111.11111111111111

Средний безубыточный объем каждой скважины больше средних предказанных значений по регионам

## Расчёт прибыли и рисков 

In [116]:
def income(data):
    product_sum = data.sort_values(by = 'product_predicted', ascending=False).head(200)['product'].sum()
    revenue = product_sum * PRICE_PER_PRODUCT
    income = revenue - BUDGET
    return income

In [117]:
def bootstrap(data):
    values = []
    for i in range(1000):
        subsample = data.sample(n = 500, replace=True, random_state=state)
        subsample = income(subsample)
        values.append(subsample)
        values_series = pd.Series(values)
        risk =  values_series.where(values_series < 0).count() / values_series.shape[0]
    return values_series.mean() , values_series.quantile(0.025), values_series.quantile(0.975), risk 

In [118]:
income0, quantile_lower0, quantile_upper0, risk0 = bootstrap(df0_predicted)
income1, quantile_lower1, quantile_upper1, risk1 = bootstrap(df1_predicted)
income2, quantile_lower2, quantile_upper2, risk2 = bootstrap(df2_predicted)

In [119]:
print(f'Регион 0: средняя рибыль = {income0/10**9:6.2f} млрд.руб. | 95% доверительынй интервал = {quantile_lower0/10**9:6.2f} млрд.руб. <{quantile_upper0/10**9:6.2f} млрд.руб. | вероятность убытка = {risk0:4.2%}')
print(f'Регион 1: средняя рибыль = {income1/10**9:6.2f} млрд.руб. | 95% доверительынй интервал = {quantile_lower1/10**9:6.2f} млрд.руб. <{quantile_upper1/10**9:6.2f} млрд.руб. | вероятность убытка = {risk1:4.2%}')
print(f'Регион 2: средняя рибыль = {income2/10**9:6.2f} млрд.руб. | 95% доверительынй интервал = {quantile_lower2/10**9:6.2f} млрд.руб. <{quantile_upper2/10**9:6.2f} млрд.руб. | вероятность убытка = {risk2:4.2%}')

Регион 0: средняя рибыль =   0.33 млрд.руб. | 95% доверительынй интервал =  -0.17 млрд.руб. <  0.86 млрд.руб. | вероятность убытка = 11.70%
Регион 1: средняя рибыль =   0.44 млрд.руб. | 95% доверительынй интервал =   0.07 млрд.руб. <  0.84 млрд.руб. | вероятность убытка = 0.90%
Регион 2: средняя рибыль =   0.37 млрд.руб. | 95% доверительынй интервал =  -0.17 млрд.руб. <  0.89 млрд.руб. | вероятность убытка = 8.30%


Исходя из представленных результатов выше, лучший регион для разработки первый.

## Вывод: 
<div class="alert alert-info"; style="border: 3px solid green">
Наилучший регион для разработки - 1. Средняя прибыль составляет 0.44 млрд.руб. с доверительным 95% интервалом от 0.04 млрд.руб до 0.84 млрд.руб с вероятностью убытка в 0.90%
</div>  